In [1]:
import pandas as pd
import pandas_profiling

import matplotlib.pyplot as plt
%matplotlib inline

# Datos Población

In [2]:
poblacion_df = pd.read_csv( './data/población.csv', sep = ';', engine = 'python' )

In [3]:
poblacion_df.dtypes

Ciudad    object
2013       int64
2014       int64
2015       int64
2016       int64
2017       int64
dtype: object

In [4]:
poblacion_df.head()

,Ciudad,2013,2014,2015,2016,2017
0,BOGOTA,7674366,7776845,7878783,7980001,8080734
1,MEDELLIN,2417325,2441123,2464322,2486723,2508452
2,CALI,2319684,2344734,2369821,2394925,2420114
3,BARRANQUILLA,1206946,1212943,1218475,1223616,1228271


In [5]:
poblacion_df = poblacion_df.set_index( 'Ciudad' ).stack().reset_index().rename( columns = { 'level_1' : 'Año', 0 : 'Población' } )

In [6]:
poblacion_df.head()

,Ciudad,Año,Población
0,BOGOTA,2013,7674366
1,BOGOTA,2014,7776845
2,BOGOTA,2015,7878783
3,BOGOTA,2016,7980001
4,BOGOTA,2017,8080734


# Datos Accidentalidad

In [7]:
accidentalidad_17_df = pd.read_csv( './data/Registro_Nacional_de_accidentalidad_2017.csv', dtype = str )
accidentalidad_16_df = pd.read_csv( './data/Registro_Nacional_de_accidentalidad_2016.csv', dtype = str )
accidentalidad_15_df = pd.read_csv( './data/Registro_Nacional_de_accidentalidad_2015.csv', dtype = str )
accidentalidad_14_df = pd.read_csv( './data/Registro_Nacional_de_accidentalidad_2014.csv', dtype = str )
accidentalidad_13_df = pd.read_csv( './data/Registro_Nacional_de_accidentalidad_2013.csv', dtype = str )

In [8]:
print( accidentalidad_17_df.shape )
print( accidentalidad_16_df.shape )
print( accidentalidad_15_df.shape )
print( accidentalidad_14_df.shape )
print( accidentalidad_13_df.shape )

(221803, 8)
(236677, 8)
(232250, 8)
(160543, 8)
(183282, 8)


In [9]:
accidentalidad_df = pd.concat( [ accidentalidad_13_df,
                               accidentalidad_14_df,
                               accidentalidad_15_df,
                               accidentalidad_16_df,
                               accidentalidad_17_df ] )

In [10]:
#accidentalidad_df[ 'Ciudad de Registro de IPAT' ].value_counts( dropna = False ).head( 20 )

In [11]:
accidentalidad_df = accidentalidad_df.loc[ accidentalidad_df[ 'Ciudad de Registro de IPAT' ].isin( ['BOGOTA', 'MEDELLIN', 'CALI', 'BARRANQUILLA' ] ) ]

In [12]:
#accidentalidad_df[ 'Fecha' ] = pd.to_datetime( ( accidentalidad_df[ 'Año Accidente' ] + '-' +
#                                               accidentalidad_df[ 'Mes Accidente' ] + '-' +
#                                               accidentalidad_df[ 'Dia Accidente' ] ), format='%Y-%m-%d' )

In [13]:
#del accidentalidad_df[ 'Año Accidente' ]
del accidentalidad_df[ 'Mes Accidente' ]
del accidentalidad_df[ 'Dia Accidente' ]
del accidentalidad_df[ 'Area del Accidente' ]
del accidentalidad_df[ 'Clase de Accidente' ]
del accidentalidad_df[ 'Clase de Choque' ]
del accidentalidad_df[ 'Gravedad del Accidente' ]

In [14]:
accidentalidad_df.rename( columns = { 'Ciudad de Registro de IPAT' : 'Ciudad', 'Año Accidente' : 'Año' }, inplace = True )

In [15]:
accidentalidad_df.dtypes

Año       object
Ciudad    object
dtype: object

In [16]:
#accidentalidad_df[ 'Fecha' ] = accidentalidad_df[ 'Fecha' ].apply( lambda x: x.replace( day = 1 ) )

In [17]:
accidentalidad_df.head()

,Año,Ciudad
7,2013,MEDELLIN
8,2013,MEDELLIN
15,2013,BARRANQUILLA
16,2013,BARRANQUILLA
17,2013,BARRANQUILLA


In [18]:
accidentalidad_df[ 'Cantidad' ] = 1

In [19]:
grouped_accidentalidad_df = accidentalidad_df.groupby( [ accidentalidad_df[ 'Ciudad' ], accidentalidad_df[ 'Año' ] ] ).agg( { 'Cantidad' : 'sum' } ).reset_index()

In [20]:
grouped_accidentalidad_df.dtypes

Ciudad      object
Año         object
Cantidad     int64
dtype: object

In [21]:
grouped_accidentalidad_df.head()

,Ciudad,Año,Cantidad
0,BARRANQUILLA,2013,5265
1,BARRANQUILLA,2014,5941
2,BARRANQUILLA,2015,6857
3,BARRANQUILLA,2016,7161
4,BARRANQUILLA,2017,7114


In [22]:
grouped_accidentalidad_df.shape

(20, 3)

# Normalización

In [23]:
grouped_accidentalidad_df = grouped_accidentalidad_df.merge( poblacion_df, how = 'left', on = [ 'Ciudad', 'Año' ] )

In [24]:
def cantidad_norm( x ):
    return x[ 'Cantidad' ] * 100000 / x[ 'Población' ]

grouped_accidentalidad_df[ 'Cantidad_norm' ] = grouped_accidentalidad_df.apply( lambda x: cantidad_norm( x ), axis = 1 )

In [25]:
#del grouped_accidentalidad_df[ '2013' ]
#del grouped_accidentalidad_df[ '2014' ]
#del grouped_accidentalidad_df[ '2015' ]
#del grouped_accidentalidad_df[ '2016' ]
#del grouped_accidentalidad_df[ '2017' ]

In [26]:
grouped_accidentalidad_df.head()

,Ciudad,Año,Cantidad,Población,Cantidad_norm
0,BARRANQUILLA,2013,5265,1206946,436.224984
1,BARRANQUILLA,2014,5941,1212943,489.800428
2,BARRANQUILLA,2015,6857,1218475,562.752621
3,BARRANQUILLA,2016,7161,1223616,585.232622
4,BARRANQUILLA,2017,7114,1228271,579.188143


# Unstacking

In [27]:
grouped_accidentalidad_df = grouped_accidentalidad_df[ [ 'Ciudad', 'Año', 'Cantidad_norm' ] ].set_index( [ 'Año', 'Ciudad' ] ).unstack( level = -1 )

In [29]:
grouped_accidentalidad_df.columns = grouped_accidentalidad_df.columns.map( '_'.join )

In [31]:
grouped_accidentalidad_df.rename( columns = { 'Cantidad_norm_BARRANQUILLA' : 'BARRANQUILLA', 
                                            'Cantidad_norm_BOGOTA' : 'BOGOTA',
                                            'Cantidad_norm_CALI' : 'CALI',
                                            'Cantidad_norm_MEDELLIN' : 'MEDELLIN' }, inplace = True )

In [32]:
grouped_accidentalidad_df = grouped_accidentalidad_df.reset_index()

In [33]:
grouped_accidentalidad_df.head()

,Año,BARRANQUILLA,BOGOTA,CALI,MEDELLIN
0,2013,436.224984,423.904724,952.327990,1897.634782
1,2014,489.800428,230.903406,264.166426,2246.302214
2,2015,562.752621,525.550710,749.676874,2311.913784
3,2016,585.232622,569.699177,731.338142,2294.867583
4,2017,579.188143,557.053357,669.844478,2145.546337


In [34]:
grouped_accidentalidad_df.to_csv( './data/accidentalidad.csv', index = False, sep = ',' )